# Git Branching: Creating Alternate Timelines

This notebook teaches you how to create, switch between, merge, and delete Git branches.

Think of branches as **parallel universes** for your code—you can experiment in one timeline without affecting the main one!

## 1. What Are Branches?

A **branch** is a pointer to a specific commit. It's like a bookmark that moves forward as you make new commits.

### Analogy: Choose Your Own Adventure Book

Imagine a story where you can:
- Keep reading the main storyline (the `main` branch)
- Fork off to explore "what if the hero went left?" (a `feature` branch)
- Come back and merge that adventure into the main story (merge)

### Why Use Branches?

1. **Experiment safely** - Try new features without breaking working code
2. **Work in parallel** - Multiple developers can work on different features
3. **Organize work** - Keep bug fixes separate from new features
4. **Review before merging** - Test changes before adding them to main code

### Basic Concept

```
main branch:
A ← B ← C ← D (main)

Create a feature branch:
        main
          ↓
A ← B ← C ← D
         ↑
      feature

After new commits on feature:
            main
              ↓
A ← B ← C ← D
         \
          E ← F (feature)
```

## 2. Setup: Create a Test Repository

Let's create a practice repo with some commits to work with.

In [ ]:
import os
import subprocess
import shutil

# Helper function to run git commands
def run_git(command, repo_path="./test_repo"):
    """Run a git command and return the output"""
    result = subprocess.run(
        f"git {command}",
        shell=True,
        cwd=repo_path,
        capture_output=True,
        text=True
    )
    output = result.stdout + result.stderr
    return output.strip()

# Clean up any existing test repo
if os.path.exists("./test_repo"):
    shutil.rmtree("./test_repo")

# Create and initialize repo
os.makedirs("./test_repo")
run_git("init")
run_git('config user.name "Tutorial User"')
run_git('config user.email "tutorial@example.com"')

print("✓ Created test repository")

In [ ]:
# Create initial commits
def create_file(filename, content):
    """Create a file in the test repo"""
    with open(f"./test_repo/{filename}", "w") as f:
        f.write(content)

# Commit 1: Initial project setup
create_file("README.md", "# My Project\n\nA simple calculator app.")
run_git("add README.md")
run_git('commit -m "Initial commit: Add README"')

# Commit 2: Add main code
create_file("calculator.py", "def add(a, b):\n    return a + b\n")
run_git("add calculator.py")
run_git('commit -m "Add addition function"')

# Commit 3: Add subtraction
with open("./test_repo/calculator.py", "a") as f:
    f.write("\ndef subtract(a, b):\n    return a - b\n")
run_git("add calculator.py")
run_git('commit -m "Add subtraction function"')

print("✓ Created 3 commits on main branch\n")
print(run_git("log --oneline"))

### Current State

```
A ← B ← C (main) (HEAD)
```

- **A** = Initial commit: Add README
- **B** = Add addition function  
- **C** = Add subtraction function
- **main** = The main branch pointer (at commit C)
- **HEAD** = "You are here" marker (currently on main)

## 3. Creating Branches

There are several ways to create branches. They all do the same thing—create a new pointer to the current commit.

### Method 1: `git branch` (create but don't switch)

In [ ]:
# Create a new branch called 'feature-multiply'
print(run_git("branch feature-multiply"))
print("\n✓ Created branch 'feature-multiply'\n")

# List all branches (* shows which one you're on)
print("All branches:")
print(run_git("branch"))

### What Happened?

```
A ← B ← C (main) (HEAD)
         ↑
   (feature-multiply)
```

- Created a new branch named `feature-multiply`
- Both `main` and `feature-multiply` point to the same commit (C)
- You're still ON the `main` branch (notice the `*` next to main)
- **HEAD** still points to `main`

Think of it like creating a new bookmark—you placed the bookmark, but you're still reading from the old page.

### Method 2: `git checkout -b` (create AND switch)

In [ ]:
# Create a new branch AND switch to it in one command
print(run_git("checkout -b feature-divide"))
print("\nCurrent branches:")
print(run_git("branch"))

### What Happened?

```
A ← B ← C (main)
         ↑
   (feature-multiply)
         ↑
   (feature-divide) (HEAD)
```

- Created `feature-divide` branch
- Switched to it (moved HEAD)
- Notice the `*` is now next to `feature-divide`
- All three branches point to the same commit (C)

This is the most common way to create branches—create and switch in one step.

### Method 3: `git switch -c` (newer, clearer syntax)

In [ ]:
# Modern alternative to 'checkout -b'
# First, go back to main
run_git("switch main")

# Create and switch to new branch
print(run_git("switch -c feature-power"))
print("\nCurrent branches:")
print(run_git("branch"))

### Which Method to Use?

| Command | What It Does | When to Use |
|---------|--------------|-------------|
| `git branch <name>` | Create only | Rarely—usually you want to switch too |
| `git checkout -b <name>` | Create and switch | Most common (works in all Git versions) |
| `git switch -c <name>` | Create and switch | Newer (Git 2.23+), clearer intent |

**Recommendation**: Use `git switch -c` for clarity, or `git checkout -b` for compatibility with older Git versions.

## 4. Visualizing Branches

Let's create a helper function to show the current branch structure.

In [ ]:
def show_branches():
    """Display branch structure with commit graph"""
    print("Branch Graph:")
    print(run_git("log --all --graph --oneline --decorate"))
    print("\nAll Branches:")
    print(run_git("branch"))

show_branches()

### Reading the Graph

- Each line is a commit
- Branches are shown in parentheses: `(HEAD -> feature-power, main, feature-multiply, feature-divide)`
- `HEAD ->` shows which branch you're currently on
- All branches currently point to the same commit (they haven't diverged yet)

## 5. Switching Branches

When you switch branches, Git:
1. Updates **HEAD** to point to the new branch
2. Updates your **working directory** to match that branch's files

### Using `git checkout` (traditional)

In [ ]:
# Switch to feature-multiply branch
print(run_git("checkout feature-multiply"))
print("\nCurrent branch:")
print(run_git("branch --show-current"))

### Using `git switch` (modern)

In [ ]:
# Switch to feature-divide branch
print(run_git("switch feature-divide"))
print("\nCurrent branch:")
print(run_git("branch --show-current"))

### What Happens to Your Files?

Let's make a change and see how files change when switching branches.

In [ ]:
# On feature-divide branch, add divide function
with open("./test_repo/calculator.py", "a") as f:
    f.write("\ndef divide(a, b):\n    return a / b\n")

run_git("add calculator.py")
run_git('commit -m "Add divide function"')

print("✓ Added divide function on feature-divide branch\n")

# Read the file
with open("./test_repo/calculator.py", "r") as f:
    print("calculator.py on feature-divide:")
    print(f.read())

In [ ]:
# Switch to main branch
run_git("switch main")

# Read the file again
with open("./test_repo/calculator.py", "r") as f:
    print("calculator.py on main:")
    print(f.read())
    
print("\n👀 Notice: The divide function disappeared!")

### What Happened?

```
            main (HEAD)
              ↓
A ← B ← C ← D
         \
          E (feature-divide)
```

- **Commit D**: Added divide function (on `feature-divide`)
- **Commit C**: Last commit on `main` (no divide function)
- When you switch branches, Git updates your files to match that branch
- The divide function exists in commit D, but not in commit C

**Analogy**: Like switching between saved games—each save has different progress.

In [ ]:
# Visualize the divergence
show_branches()

## 6. Merging Branches

Once you've finished work on a branch, you **merge** it back into main. There are two types of merges:

### Type 1: Fast-Forward Merge (Simple)

Happens when the target branch hasn't changed since you created your branch.

```
Before merge:
            main
              ↓
A ← B ← C ← D ← E (feature)

After merge:
                  main
                    ↓
A ← B ← C ← D ← E
```

Git just moves the `main` pointer forward—no new commit needed!

In [ ]:
# Make sure we're on main
run_git("switch main")

# Merge feature-divide into main
print(run_git("merge feature-divide"))
print("\n✓ Merged feature-divide into main")

# Check the file
with open("./test_repo/calculator.py", "r") as f:
    print("\ncalculator.py now has:")
    print(f.read())

In [ ]:
show_branches()

### What Happened?

- Notice "Fast-forward" in the merge message
- Git moved `main` forward to point to the same commit as `feature-divide`
- No new merge commit was created
- Both branches now point to the same commit

### Type 2: Three-Way Merge (Complex)

Happens when both branches have new commits.

```
Before merge:
            D (main)
           /
A ← B ← C
           \
            E ← F (feature)

After merge:
            D ← M (main)
           /   /
A ← B ← C   /
           \ /
            E ← F (feature)
```

Git creates a new **merge commit** (M) that has two parents (D and F).

In [ ]:
# Create changes on TWO branches to demonstrate 3-way merge

# First, add something to main
run_git("switch main")
with open("./test_repo/calculator.py", "a") as f:
    f.write("\n# Main calculator module\n")
run_git("add calculator.py")
run_git('commit -m "Add comment to main"')

# Then, add something different on feature-multiply
run_git("switch feature-multiply")
with open("./test_repo/calculator.py", "a") as f:
    f.write("\ndef multiply(a, b):\n    return a * b\n")
run_git("add calculator.py")
run_git('commit -m "Add multiply function"')

print("✓ Created divergent commits on main and feature-multiply\n")
show_branches()

### Current State

```
              D (Add comment) (main)
             /
A ← B ← C ← E (Add divide)
             \
              F (Add multiply) (feature-multiply)
```

Now let's merge `feature-multiply` into `main`.

In [ ]:
# Switch to main and merge
run_git("switch main")
print(run_git("merge feature-multiply -m 'Merge multiply feature'"))
print("\n✓ Three-way merge complete\n")

show_branches()

### What Happened?

```
After merge:
              D ← M (main)
             /   /
A ← B ← C ← E  /
             \ /
              F (feature-multiply)
```

- Git created a new **merge commit** (M)
- This commit has TWO parents: D (from main) and F (from feature-multiply)
- Git combined the changes from both branches
- The file now has BOTH the comment and the multiply function

In [ ]:
# Check the merged file
with open("./test_repo/calculator.py", "r") as f:
    print("calculator.py after merge:")
    print(f.read())
    
print("\n👀 Notice: Has BOTH the comment AND multiply function!")

## 7. Merge Conflicts

A **conflict** happens when both branches modify the SAME lines in a file.

### Why Conflicts Happen

```
main:     Changed line 5 to "hello"
feature:  Changed line 5 to "goodbye"

Git: "I don't know which one to keep—you decide!"
```

Let's create a conflict intentionally.

In [ ]:
# Create a conflict by editing the same line on two branches

# On main: Change the README
run_git("switch main")
create_file("README.md", "# My Project\n\nAn advanced calculator app.")
run_git("add README.md")
run_git('commit -m "Update README: advanced calculator"')

# On feature-power: Change the SAME line differently
run_git("switch feature-power")
create_file("README.md", "# My Project\n\nA simple calculator app with power function.")
run_git("add README.md")
run_git('commit -m "Update README: add power mention"')

print("✓ Created conflicting changes in README.md")

In [ ]:
# Try to merge—this will create a conflict!
run_git("switch main")
result = run_git("merge feature-power")
print(result)
print("\n⚠️ Conflict detected!")

### Understanding Conflict Markers

Git marks conflicts in your file with special markers:

```
<<<<<<< HEAD
Code from your current branch (main)
=======
Code from the branch you're merging (feature-power)
>>>>>>> feature-power
```

Let's look at the conflicted file.

In [ ]:
# Read the conflicted file
with open("./test_repo/README.md", "r") as f:
    content = f.read()
    print("Conflicted README.md:")
    print(content)
    print("\n👆 Notice the <<<<<<, =======, and >>>>>>> markers")

### Resolving the Conflict

To resolve:
1. **Open the file** and find the conflict markers
2. **Decide** which version to keep (or combine them)
3. **Remove** the conflict markers (`<<<<<<<`, `=======`, `>>>>>>>`)
4. **Save** the file
5. **Stage** the resolved file with `git add`
6. **Commit** to complete the merge

In [ ]:
# Resolve by combining both descriptions
resolved_content = "# My Project\n\nAn advanced calculator app with power function."
create_file("README.md", resolved_content)

print("Resolved README.md:")
print(resolved_content)
print("\n✓ Conflict resolved by combining both versions")

In [ ]:
# Complete the merge
run_git("add README.md")
print(run_git('commit -m "Merge feature-power: resolve README conflict"'))
print("\n✓ Merge complete!")

### Conflict Resolution Strategy

| Situation | Action |
|-----------|--------|
| Your version is correct | Keep the `HEAD` section, delete the other |
| Their version is correct | Keep the incoming section, delete `HEAD` |
| Both needed | Combine both sections |
| Neither correct | Write new code, delete both |

**Always**:
- Remove ALL conflict markers (`<<<<<<<`, `=======`, `>>>>>>>`)
- Test the code before committing
- Communicate with teammates if you're unsure

## 8. Deleting Branches

After merging, you can delete the branch to keep things tidy.

### Safe Delete: `git branch -d`

Only deletes if the branch has been merged.

In [ ]:
# Delete merged branches
print(run_git("branch -d feature-divide"))
print(run_git("branch -d feature-multiply"))
print(run_git("branch -d feature-power"))

print("\n✓ Deleted merged branches\n")
print("Remaining branches:")
print(run_git("branch"))

### Force Delete: `git branch -D`

Deletes even if NOT merged (use with caution!).

In [ ]:
# Create a branch with unmerged work
run_git("switch -c experimental-feature")
create_file("experiment.py", "# Experimental code\nprint('testing')")
run_git("add experiment.py")
run_git('commit -m "Add experimental feature"')

# Try safe delete (will fail)
run_git("switch main")
print("Try safe delete:")
print(run_git("branch -d experimental-feature"))

In [ ]:
# Force delete
print("Force delete:")
print(run_git("branch -D experimental-feature"))
print("\n⚠️ Branch deleted even though it wasn't merged!")

### When to Use Each

| Command | Use When | Safety |
|---------|----------|--------|
| `git branch -d` | Branch is merged | Safe—won't lose work |
| `git branch -D` | Abandoning unmerged work | Dangerous—will lose commits |

**Best Practice**: Always try `-d` first. Only use `-D` if you're certain you want to discard the work.

## 9. Practice Exercise

Create a complete feature workflow:

1. Create a branch called `feature-modulo`
2. Add a modulo function to `calculator.py`
3. Commit the change
4. Switch back to `main`
5. Merge `feature-modulo` into `main`
6. Delete the `feature-modulo` branch

Try it yourself before looking at the solution!

In [ ]:
# Your turn! Complete the exercise here

# 1. Create branch


# 2. Add modulo function
# def modulo(a, b):
#     return a % b


# 3. Commit


# 4. Switch to main


# 5. Merge


# 6. Delete branch


## 10. Solution

In [ ]:
# 1. Create and switch to feature branch
print("Step 1: Create branch")
print(run_git("switch -c feature-modulo"))
print()

In [ ]:
# 2. Add modulo function
print("Step 2: Add modulo function")
with open("./test_repo/calculator.py", "a") as f:
    f.write("\ndef modulo(a, b):\n    return a % b\n")
print("✓ Added modulo function\n")

In [ ]:
# 3. Commit the change
print("Step 3: Commit")
run_git("add calculator.py")
print(run_git('commit -m "Add modulo function"'))
print()

In [ ]:
# 4. Switch back to main
print("Step 4: Switch to main")
print(run_git("switch main"))
print()

In [ ]:
# 5. Merge feature branch
print("Step 5: Merge feature-modulo")
print(run_git("merge feature-modulo"))
print()

In [ ]:
# 6. Delete the branch
print("Step 6: Delete branch")
print(run_git("branch -d feature-modulo"))
print()

print("\n✓ Exercise complete!")
print("\nFinal branches:")
print(run_git("branch"))

In [ ]:
# Verify the final calculator has all functions
with open("./test_repo/calculator.py", "r") as f:
    print("Final calculator.py:")
    print(f.read())

## Key Takeaways

### Branch Basics
1. **Branch** = pointer to a commit (like a bookmark)
2. **HEAD** = pointer to current branch ("you are here")
3. **Branches are cheap** = create them freely for experiments

### Commands Cheat Sheet

| Task | Command | Notes |
|------|---------|-------|
| Create branch | `git branch <name>` | Doesn't switch to it |
| Create + switch | `git switch -c <name>` | Modern (Git 2.23+) |
| Create + switch | `git checkout -b <name>` | Traditional (works everywhere) |
| Switch branch | `git switch <name>` | Modern |
| Switch branch | `git checkout <name>` | Traditional |
| List branches | `git branch` | `*` shows current |
| Merge branch | `git merge <name>` | Run from target branch |
| Delete (safe) | `git branch -d <name>` | Only if merged |
| Delete (force) | `git branch -D <name>` | Even if not merged |

### Merge Types
1. **Fast-forward**: Target hasn't changed, just move pointer forward
2. **Three-way**: Both branches have changes, create merge commit

### Conflict Resolution
1. Git marks conflicts with `<<<<<<<`, `=======`, `>>>>>>>`
2. Edit file to resolve
3. Remove ALL markers
4. `git add` + `git commit` to complete merge

### Workflow Pattern
```bash
git switch -c feature-name  # Create feature branch
# ... make changes ...
git add .
git commit -m "Add feature"
git switch main             # Go back to main
git merge feature-name      # Merge feature in
git branch -d feature-name  # Clean up
```

### Best Practices
1. Create a branch for each feature/fix
2. Keep branches short-lived (merge often)
3. Use descriptive names (`feature-login`, `fix-crash`, `refactor-database`)
4. Delete merged branches to avoid clutter
5. Always commit before switching branches
6. Test after merging before pushing

### Analogies to Remember
- **Branch** = alternate timeline / parallel universe
- **Merge** = combining timelines back together
- **HEAD** = "you are here" marker on a map
- **Conflict** = Git can't decide, needs human judgment
- **Fast-forward** = no divergence, just move forward in time